# Contexto

O **tipo** Contexto é definido para transporta prazos, sinais de cancelamento, e outros valores do **escopo de uma requisição** entre fronteiras de uma API e **entre processos**.

Exemplo: Uma chamada a um servidor HTTP inicia um contexto, este contexto é propagado através das chamadas a serviços externos ou banco de dados. Se gorrotinas forem lançadas, o contexto deve ser propagado para elas.

Esta cadeia de chamadas que ocorre deve propagar o contexto, podendo substitui-lo por um contexto derivado com sinal de cancelamento (WithCancel), com prazos (WithTimeout, WithDeadline) ou carregando valores (WithValue).

É definida através de uma **interface** que define os seguintes métodos:


`Deadline() (deadline time.Time, ok bool)` 

Retorna o momento (deadline) que o contexto deve ser cancelado. 
Contextos que não tenham prazo retornam ok como false.


`Done() <-chan struct{}` 

Done devolve um canal que é fechado quando o trabalho efetuado em nome deste contexto deve ser cancelado.
Quando um cancelamento for invocado ou um prazo for atingido o canal deve ser cancelado.


`Err() error` 

Se o canal **Done** não foi fechado ainda então `nil` é retornado, senão, um erro é retornado explicando o porquê. 

`Value(key any) any` 

Devolve o valor associado a este contexto para key, ou nil se não existir nenhum valor associado a key.
Obs: não utilize para passar parâmetros opicionais para funções.


Fonte: https://cs.opensource.google/go/go/+/refs/tags/go1.24.2:src/context/context.go

In [29]:
// ...
type Context interface {
	Deadline() (deadline time.Time, ok bool)
	Done() <-chan struct{}
	Err() error
	Value(key any) any
}
// ...

## Qual a diferença entre context.TODO e context.Background

**Resposta**

O *context.TODO* e *Context.Background* são idênticos e possuem a mesma base comum que é um contexto vazio (*emptyCtx*). Um contexto vazio não é cancelável, não possui valores e não possui prazo (deadline).

Utilizamos o *TODO* quando não é claro qual o contexto a utilizar ou quando este ainda não está disponível, já o Background é tipicamente utilizado pela função principal, inicialização e testes, e como contexto de nível superior para requisições recebidas.

Fonte: https://cs.opensource.google/go/go/+/refs/tags/go1.24.2:src/context/context.go;l=179-225

## Passagem de valor em um contexto

Um contexto pode carregar um valor associado a uma chave, por exemplo um id de correlação utilizado em rastros distribuídos ou um usuário do sistema.

In [30]:
// Não exportado para previnir colisões com chaves definidas em outros pacotes
type contextKey string
var  correlationIDKey = contextKey("correlationID")
%%
ctx := context.WithValue(context.Background(), correlationIDKey, "0196845e-3337-71e2-bd00-523181774dea")

// É necessário a conversão do valor para seu tipo original
fmt.Printf("A chave de correlação é %s", ctx.Value(correlationIDKey).(string))

A chave de correlação é 0196845e-3337-71e2-bd00-523181774dea

In [31]:
// User é o tipo de valor armazenado nos Contexts.
type User struct {
    name string
}
	
// key é um tipo não exportado para chaves definidas neste pacote.
// Isso previne colisões com chaves definidas em outros pacotes.
type key int

// userKey é a chave para valores do tipo user.User em Contexts. Ela é
// não exportada; os clientes devem usar user.NewContext e user.FromContext
// ao invés de usar essa chave diretamente.
var userKey key

// NewContext retorna um novo Context que carrega o valor u.
func NewContext(ctx context.Context, u *User) context.Context {
    return context.WithValue(ctx, userKey, u)
}

// FromContext retorna o valor User armazenado em ctx, se houver.
func FromContext(ctx context.Context) (*User, bool) {
    u, ok := ctx.Value(userKey).(*User)
    return u, ok
}

%%
ctx := NewContext(context.Background(), &User{name: "João"})
user, ok := FromContext(ctx)
if ok{
    fmt.Println(user.name)    
}


João


## Cancelamento

Um contexto com cancelamento, possui uma função de cancelamento que indica que o trabalho executado em seu contexto deve ser interrompido.

Após a primeira chamada, as subsequentes a função de cancelamento não irão fazer nada.

In [39]:
%%
// gen gera números inteiros em uma goroutine separada e
// os envia para o canal retornado.
// Os chamadores de gen precisam cancelar o contexto assim que
// terminarem de consumir os números inteiros gerados, para não vazar
// a goroutine interna iniciada por gen.
gen := func(ctx context.Context) <-chan int {
	dst := make(chan int)
	n := 1
	go func() {
        defer close(dst)
		for {
			select {
			case <-ctx.Done():
				return // retorna para não vazar a goroutine
			case dst <- n:
				n++
			}
		}
	}()
	return dst
}

ctx, cancel := context.WithCancel(context.Background())
defer cancel() // cancelar quando terminarmos de consumir os inteiros

for n := range gen(ctx) {
	fmt.Println(n)
    // Descomente para cancelar o contexto antes da hora
    //if n == 3 {
    //    cancel() 
    //}
	if n == 5 {
		break
	}
}

1
2
3
4
5


## Qual a diferença de timeout e deadline?

**Resposta**

O contexto `WithTimeout` na verdade é um `WithDeadline` onde o prazo (deadline) é o momento atual incremento do **timeout**.

O contexto com prazo (WithDeadline) quando atinge o seu momento, realiza o cancelamento do contexto fechando o canal `Done`.

Use WithTimeout se você quer algo como "me dê a resposta em até X segundos",e WithDeadline se você quer algo como "me dê a resposta até exatamente TAL hora".

Fonte: https://cs.opensource.google/go/go/+/refs/tags/go1.24.2:src/context/context.go;l=694-696

In [49]:
var ( 
    neverReady = make(chan struct{}) // jamais fechado
    shortDuration    = 1 * time.Millisecond
)
%%
d := time.Now().Add(shortDuration)
ctx, cancel := context.WithDeadline(context.Background(), d)

// Mesmo que ctx vá expirar, é uma boa prática chamar sua função
// de cancelamento de qualquer forma. Deixar de fazer isso pode manter
// o contexto e seu pai vivos por mais tempo do que o necessário.
defer cancel()

select {
case <-neverReady:
    fmt.Println("ready")
case <-ctx.Done():
    fmt.Println(ctx.Err())
}

context deadline exceeded


In [50]:
%%
ctx, cancel := context.WithTimeout(context.Background(), shortDuration)

// Mesmo que ctx vá expirar, é uma boa prática chamar sua função
// de cancelamento de qualquer forma. Deixar de fazer isso pode manter
// o contexto e seu pai vivos por mais tempo do que o necessário.
defer cancel()

select {
case <-neverReady:
    fmt.Println("ready")
case <-ctx.Done():
    fmt.Println(ctx.Err())
}

context deadline exceeded


## Causa de um cancelamento

Quando um contexto é cancelado, o erro indica que um contexto foi cancelado, mas não especifica a razão exata.

Desde a versão 1.21, agora podemos criar um contexto usando `context.WithCancelCause()`. A função de cancelamento `cancel` irá aceitar um único parâmetro que é a causa raiz do cancelamento (error).

Existe uma variante com causa para os contextos canceláveis: `WithCancelCause`, `WithTimeoutCause` e `WithDeadlineCause`.

In [56]:
%%
ctx, cancel := context.WithCancelCause(context.Background())
cancel(errors.New("the night is dark"))

fmt.Println("Erro: ", ctx.Err())
fmt.Println("Causa: ", context.Cause(ctx))

Erro:  context canceled
Causa:  the night is dark


In [58]:
%%
cause := errors.New("the night is dark")
ctx, cancel := context.WithTimeoutCause(
    context.Background(), 10*time.Millisecond, cause,
)
defer cancel()

time.Sleep(50 * time.Millisecond)
fmt.Println(ctx.Err())
// context deadline exceeded
fmt.Println(context.Cause(ctx))
// the night is dark

context deadline exceeded
the night is dark


## Propagação de contexto

O contexto deve ser propagado ao longo da cadeia de chamadas para que todas as funções envolvidas possam respeitar cancelamentos, prazos e valores associados à mesma operação.

A propagação de contexto em Go permite que contextos derivados compartilhem o mesmo ciclo de vida de um contexto superior. Quando o contexto superior é cancelado — seja manualmente com cancel() ou automaticamente por timeout ou deadline — todos os contextos derivados também são cancelados automaticamente. Isso facilita o controle e a finalização coordenada de operações assíncronas, evitando vazamento de recursos como goroutines ou conexões abertas. Esse mecanismo é especialmente útil em aplicações concorrentes que precisam interromper múltiplas tarefas relacionadas de forma segura e eficiente.

O cancelamento de um contexto derivado não afeta o contexto superior, pois a propagação ocorre apenas do superior para os derivados, e não no sentido inverso.

In [59]:
%%

// Criando o contexto superior com cancelamento
ctxSuperior, cancel := context.WithCancel(context.Background())

// Criando dois contextos derivados do contexto superior
ctxDerivado1, _ := context.WithCancel(ctxSuperior)
ctxDerivado2, _ := context.WithCancel(ctxSuperior)

// Função para simular trabalho com um contexto
doWork := func(ctx context.Context, nome string) {
    select {
    case <-ctx.Done():
        fmt.Printf("%s cancelado: %v\n", nome, ctx.Err())
    case <-time.After(3 * time.Second):
        fmt.Printf("%s completou sem cancelamento\n", nome)
    }
}

// Iniciando goroutines com os contextos derivados
go doWork(ctxDerivado1, "Contexto Derivado 1")
go doWork(ctxDerivado2, "Contexto Derivado 2")

// Espera 1 segundo e então cancela o contexto superior
time.Sleep(1 * time.Second)
fmt.Println("Cancelando o contexto superior...")
cancel()

// Aguarda para que as goroutines imprimam o cancelamento
time.Sleep(1 * time.Second)



Cancelando o contexto superior...
Contexto Derivado 2 cancelado: context canceled
Contexto Derivado 1 cancelado: context canceled


## Dicas

* Não armazene Contexts dentro de um tipo struct; em vez disso, passe o Context explicitamente para cada função que precisar dele. Isso é discutido mais a fundo em https://go.dev/blog/context-and-structs.
* Não passe um Context nulo (nil), mesmo que uma função permita isso. Passe context.TODO se você não tiver certeza de qual Context usar.
* Use os valores do context apenas para dados com escopo de requisição que transitam entre processos e APIs, e não para passar parâmetros opcionais para funções.
* O mesmo Context pode ser passado para funções executando em diferentes goroutines; Contexts são seguros para uso simultâneo por múltiplas goroutines.
* O Context deve ser o primeiro parâmetro, normalmente nomeado como ctx: func DoSomething(ctx context.Context, arg Arg) error 

## Exemplos

### Stream

```go
    // Stream gera valores com DoSomething e os envia para out
    // até que DoSomething retorne um erro ou ctx.Done seja fechado.
	func Stream(ctx context.Context, out chan<- Value) error {
	  	for {
	  		v, err := DoSomething(ctx)
	  		if err != nil {
	  			return err
	  		}
	  		select {
	  		case <-ctx.Done():
	  			return ctx.Err()
	  		case out <- v:
	  		}
	  	}
	}
```

### Fan-out
```go
// publicar tenta enviar um valor para o canal `saida` e utiliza um contexto com timeout
// para garantir que a operação não dure mais do que o tempo especificado.
func publicar(ctx context.Context, saida chan<- int, valor int, controle chan<- struct{}) {
	// Cria um contexto com timeout de 1 segundo
	ctx, cancel := context.WithTimeout(ctx, 1*time.Second)
	defer cancel()

	select {
	case <-ctx.Done():
		// Se o contexto expirar antes do envio, não faz nada
	case saida <- valor:
		// Se o valor for enviado com sucesso antes do timeout
	}
	controle <- struct{}{}
}

func fanout(entrada <-chan int, saidas ...chan<- int) {
	// Canal para controlar o término das publicações
	controle := make(chan struct{}, len(saidas)*2) // capacidade para controle de todas as publicações

	for valor := range entrada {
		// Publica o valor de entrada em todas as saídas
		for _, saida := range saidas {
			go publicar(context.Background(), saida, valor, controle)
		}
		// Aguarda o término de todas as publicações
		for i := 0; i < len(saidas); i++ {
			<-controle
		}
	}
	// Como a entrada foi consumida, fecha os canais de saída
	for _, saida := range saidas {
		close(saida)
	}
}

```

## Referências:
* https://antonz.org/go-concurrency/context/
* https://cs.opensource.google/go/go/+/refs/tags/go1.24.2:src/context/context.go
* https://pkg.go.dev/context